In [1]:
%pip install -q -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext autoreload
%autoreload 1
from imports import *
from functions import *

In [3]:
import dask.dataframe as dd

# get all stations and some metadata as a Dask DataFrame
stations_df = api.stations()
# parse the response as a Dask DataFrame
stations_df = dd.from_pandas(api.stations(as_df=True), npartitions=4)

print(len(stations_df))

146


# Filter Buoys by Remarks

In [4]:
import dask.dataframe as dd
import numpy as np

access_error_url_list = []

# Liste de mots à rechercher dans la colonne "Remark"
blacklist = ["Failure", "ceased", "failed", "recovered", "stopped", 'adrift']
stations_id_set = set()

print(f'Avant Filtre: {stations_df.shape[0].compute()}')  # Utiliser .compute() pour évaluer l'expression et obtenir le nombre réel

# Liste pour collecter les indices à supprimer
indices_a_supprimer = []

# Fonction qui sera appliquée à chaque partition
def filter_rows(df):
    indices_a_supprimer_partition = []
    for idx, row in df.iterrows():
        station_id = row["Station"]
        station_Location = row["Hull No./Config and Location"]  # Extraire la valeur de la cellule pour chaque ligne

        # Vérifier si station_Location n'est pas NaN
        if isinstance(station_Location, str) and ")" in station_Location:
            station_name = station_Location.split(')')[1].rstrip(" )")  # On enlève l'espace et la parenthèse en fin de chaîne
        else:
            station_name = station_Location.strip() if isinstance(station_Location, str) else ""  # Si pas de ")", on garde toute la chaîne

        station_name = station_name.rstrip(" )").replace("(", "").replace(")", "").strip()

        # Nettoyage final pour enlever toute parenthèse ou espace en fin de nom
        station_name = station_name.rstrip(" )")

        # Vérifier si "Remark" n'est pas NaN et si un des éléments de blacklist est dans "Remark"
        if isinstance(row["Remark"], str) and any(blacklist_word.lower() in row["Remark"].lower() for blacklist_word in blacklist):
            # Ajouter l'index à la liste
            indices_a_supprimer_partition.append(idx)
            url = get_buoy_url(station_id)
            access_error_url_list.append(url)
    
    # Retourner les lignes restantes après suppression
    return df.loc[~df.index.isin(indices_a_supprimer_partition)]

# Définir un meta (structure de données pour Dask)
meta = stations_df.head(0)  # Cela va retourner la structure des colonnes sans les données.

# Appliquer le filtre sur toutes les partitions
stations_df = stations_df.map_partitions(filter_rows, meta=meta)

# Utiliser .compute() pour obtenir la taille réelle après le filtrage
print(f'Après Filtre: {stations_df.shape[0].compute()}')  # Utiliser .compute() ici aussi pour évaluer le résultat


Avant Filtre: 146
Après Filtre: 43


# Build Buoys_datas Dict

In [5]:
# Dictionnaire pour stocker les DataFrames, clé : ID de la bouée, valeur : DataFrame
buoy_datas = {}
buoy_list = []

# Parcours de chaque bouée dans stations_df
for index, row in stations_df.iterrows():
    buoy_id = row['Station']
    metadata = get_station_metadata(buoy_id)

    # ✅ Récupérer les données sous forme de dictionnaire
    buoy_info = parse_buoy_json(metadata)

    # ✅ Stocker directement les données dans buoy_datas
    buoy_datas[buoy_id] = buoy_info
    buoy_list.append(buoy_id)

# Affichage du nombre de bouées réussies et échouées
print(f"Nombre de bouées traitées : {len(buoy_datas)}\n")

# Afficher le contenu de buoy_datas

first_key = next(iter(buoy_datas))
first_key
buoy_datas[first_key]


🔍 Début du parsing de la bouée...
🌍 Zone de la station : southwest pass, la
🆔 Station ID : BURL1
✅ Coordonnées extraites : Latitude = 28.91N, Longitude = 89.43W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation (m): 13.4
💨 Anemometer Height : 38
🌤️ Air Temp Height : 13.7
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=BURL1
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !


🔍 Début du parsing de la bouée...
🌍 Zone de la station : five fingers, ak
🆔 Station ID : FFIA2
✅ Coordonnées extraites : Latitude = 57.27N, Longitude = 133.63W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation (m):

{'station_zone': 'southwest pass, la',
 'lat_buoy': '28.91N',
 'lon_buoy': '89.43W',
 'Water_depth': 'N/A',
 'sea_temp_depth': None,
 'Barometer_elevation': '13.4',
 'Anemometer_height': '38',
 'Air_temp_height': '13.7',
 'url': 'https://www.ndbc.noaa.gov/station_page.php?station=BURL1'}

# Marine and Meteo Data Collection Process

In [6]:
# 🚀 Démarrage du processus
print("\n🚀 Démarrage du processus de collecte des données...\n")

# Initialisation des compteurs
marine_data_collected_successfully = marine_data_collected_failed = 0
meteo_data_collected_successfully = meteo_data_collected_failed = 0

success = False
total_stations = stations_df.shape[0]
count = 0

# 🔄 Parcours des bouées / stations
for idx, row in stations_df.iterrows():
    buoy_id = row["Station"]

    ######### 🌊 MARINE DATA #########
    try:
        df_marine = NDBC.realtime_observations(buoy_id)
        if df_marine is None or df_marine.empty:
            marine_data_collected_failed += 1
            continue

        marine_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    # Ajout des métadonnées
    try:
        buoy_info = buoy_datas.get(buoy_id, {})
        Lat, Lon = buoy_info.get('lat_buoy'), buoy_info.get('lon_buoy')
        if Lat is None or Lon is None:
            raise ValueError(f"Données manquantes pour {buoy_id}")

        df_marine['Lat'] = Lat
        df_marine['Lon'] = Lon
        df_marine['Water_depth'] = buoy_info.get('Water_depth', None)
        df_marine.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_marine.columns]
        df_marine['Datetime'] = df_marine['Datetime'].dt.tz_localize(None)

        buoy_datas[buoy_id]["Marine"] = df_marine

        station_zone = safe_get(parse_buoy_json(get_station_metadata(buoy_id)), "station_zone")
        Bronze_Marine_table_Name = f"br_{buoy_id}_marine_{station_zone}".replace('.', '_').replace('-', '_').replace(' ', '_').lower()

    except Exception as e:
        print(f"⚠️ Erreur métadonnées marine {buoy_id}: {e}")
        marine_data_collected_failed += 1
        continue

    ######### ⛅ METEO DATA #########
    try:
        df_meteo = meteo_api_request([Lat, Lon])
        if df_meteo is None or df_meteo.empty:
            meteo_data_collected_failed += 1
            continue
        
        rename_columns(df_meteo, {'date':'Datetime'})
        df_meteo.columns = ['Datetime' if 'date' in col.lower() or 'time' in col.lower() else col for col in df_meteo.columns]
        df_meteo['Datetime'] = df_meteo['Datetime'].dt.tz_localize(None)
    
        buoy_datas[buoy_id]["Meteo"] = df_meteo
        meteo_data_collected_successfully += 1
    except Exception as e:
        print(f"⚠️ Erreur collecte météo {buoy_id}: {e}")
        meteo_data_collected_failed += 1
        continue

# Retirer les bouées avec des DataFrames vides ou None
buoy_datas = {buoy_id: data for buoy_id, data in buoy_datas.items() 
              if "Marine" in data and data["Marine"] is not None and not data["Marine"].empty
              and "Meteo" in data and data["Meteo"] is not None and not data["Meteo"].empty}

# 🔚 Résumé final

print("\n📝 Résumé final :")
print(f"🌊 Marine - Collecte ✅ {marine_data_collected_successfully} ❌ {marine_data_collected_failed}")
print(f"⛅ Météo - Collecte ✅ {meteo_data_collected_successfully} ❌ {meteo_data_collected_failed}")

# Afficher la longueur du dictionnaire (nombre de bouées avec des données valides)
print(f"\n📊 Nombre de bouées avec des données valides : {len(buoy_datas)}")


🚀 Démarrage du processus de collecte des données...


🔍 Début du parsing de la bouée...
🌍 Zone de la station : southwest pass, la
🆔 Station ID : BURL1
✅ Coordonnées extraites : Latitude = 28.91N, Longitude = 89.43W
🌊 Water Depth : N/A
🌡️ Sea Temp Depth : None
🌬️ Barometer Elevation (m): 13.4
💨 Anemometer Height : 38
🌤️ Air Temp Height : 13.7
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=BURL1
✅ Parsing terminé !

📊 station_zone : southwest pass, la
🔄 Colonne 'date' renommée en 'Datetime'
✅ Colonnes renommées : {'date': 'Datetime'}

🔍 Début du parsing de la bouée...
🌍 Zone de la station : grays reef
🆔 Station ID : 41008
✅ Coordonnées extraites : Latitude = 31.40N, Longitude = 80.87W
🌊 Water Depth : 16 m
🌡️ Sea Temp Depth : 2
🌬️ Barometer Elevation (m): 2.4
💨 Anemometer Height : 3.8
🌤️ Air Temp Height : 3.4
🔗 URL de la bouée : https://www.ndbc.noaa.gov/station_page.php?station=41008
✅ Parsing terminé !

📊 station_zone : grays reef
🔄 Colonne 'date' renommée en 'D

In [7]:
display_buoys_missing_df_counts(buoy_datas)


🌊 Nombre de bouées sans données 'Marine' : 0/41

☁️ Nombre de bouées sans données 'Meteo' : 0/41


# Data Enrichment with MetaDatas

In [ ]:
# Liste des colonnes à ne pas inclure dans l'ajout
list_not_include = ['lon_buoy', "lat_buoy", "url"]

# Parcours des bouées
for buoy_id, value in buoy_datas.items():
    print(f"\n🔍 Traitement de la Station ID: {buoy_id}")

    marine_df = buoy_datas[buoy_id]["Marine"]
    meteo_df = buoy_datas[buoy_id]["Meteo"]

    try:
        # Récupérer les métadonnées de la station
        buoy_metadata = get_station_metadata(buoy_id)
        parsed_data = parse_buoy_json(buoy_metadata)

        # Mise à jour du dictionnaire avec les métadonnées
        data = buoy_datas[buoy_id]
        data.update(parsed_data)
        
        # Si marine_df est un DataFrame Pandas, on le convertit en Dask
        if marine_df is not None:
            # Vérifier si le DataFrame est valide
            if not marine_df.empty:
                marine_df = dd.from_pandas(marine_df, npartitions=4)
                marine_df["Station ID"] = str(buoy_id)  # Ajout de la station ID

                # Ajouter les métadonnées
                for key, value in parsed_data.items():
                    if key not in list_not_include:
                        marine_df[key] = value
                        print(f"✅ Colonne '{key}' ajoutée au DataFrame de la station {buoy_id}")

                # Mise à jour de la bouée avec le DataFrame Dask
                buoy_datas[buoy_id]["Marine"] = marine_df

    except Exception as e:
        print(f"❌ Erreur pour la station {buoy_id}: {e}")

# Vérification de l'ajout des colonnes en prenant un id au hasard
station_id = random.choice(list(buoy_datas.keys()))
marine_df = buoy_datas[station_id]["Marine"]

if marine_df is not None:
    print("\nColonnes ajoutées au DataFrame de la station", station_id)
    print(marine_df.columns.to_list())  # Affiche les colonnes sans faire de .compute()



🔍 Traitement de la Station ID: BURL1


RuntimeError: Failed to generate metadata for df. This operation may not be supported by the current backend.

In [ ]:
display_row_values(df_marine)

In [ ]:
display(df_marine.columns)
display(df_meteo.columns)

# Handle Null Values

In [ ]:
important_columns_oceanography = [
    'wind_direction',             
    'wind_speed',                 
    'wave_height',                   
    'pressure',                   
    'air_temperature',            
    'water_temperature',          
    'Datetime',
    'Lat',
    'Lon'                 
]

important_columns_meteorology = [
    'temperature_2m',             
    'relative_humidity_2m',       
    'dew_point_2m',               
    'precipitation',              
    'pressure_msl',               
    'cloud_cover',                
    'wind_speed_10m',             
    'Datetime'
]

stations_depart = len(buoy_datas)
ignored_buoys = {}  # Dictionary to track ignored buoys and their reasons

for station_id, data in buoy_datas.items():
    print(f"\n🔄 Nettoyage des données pour la station {station_id}")

    marine_df = data.get("Marine")
    meteo_df = data.get("Meteo")

    if marine_df is None or meteo_df is None:
        ignored_buoys[station_id] = "Marine DataFrame ou Meteo DataFrame manquant(e)"
        print(f"⚠️ Station {station_id} ignorée: Marine DataFrame ou Meteo DataFrame manquant(e)")
        continue

    try:
        # Convertir les pandas DataFrames en Dask DataFrames
        marine_df = dd.from_pandas(marine_df, npartitions=4)
        meteo_df = dd.from_pandas(meteo_df, npartitions=4)

        # Nettoyage des DataFrames
        cleaned_marine_df = handle_null_values(marine_df)
        cleaned_meteo_df = handle_null_values(meteo_df)

        # Vérification des colonnes importantes après nettoyage
        marine_columns_ok = all(col in cleaned_marine_df.columns for col in important_columns_oceanography)
        meteo_columns_ok = all(col in cleaned_meteo_df.columns for col in important_columns_meteorology)

        # Track which columns are missing
        missing_marine_columns = [col for col in important_columns_oceanography if col not in cleaned_marine_df.columns]
        missing_meteo_columns = [col for col in important_columns_meteorology if col not in cleaned_meteo_df.columns]

        if missing_marine_columns or missing_meteo_columns:
            ignored_buoys[station_id] = f"Colonnes manquantes: Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}"
            print(f"⚠️ Station {station_id} ignorée: Colonnes manquantes - Marine: {missing_marine_columns}, Meteo: {missing_meteo_columns}")
            continue

        # Ajouter le DataFrame nettoyé au dictionnaire des résultats
        buoy_datas[station_id]['Cleaned Marine'] = cleaned_marine_df
        buoy_datas[station_id]['Cleaned Meteo'] = cleaned_meteo_df
        print(f"✅ Nettoyage réussi pour la station {station_id} ({cleaned_marine_df.shape[0].compute()} lignes)")

    except Exception as e:
        ignored_buoys[station_id] = f"Erreur lors du nettoyage: {e}"
        print(f"❌ Erreur lors du nettoyage pour {station_id}: {e}")

# 🔥 Suppression des stations ignorées du dictionnaire principal
for station_id in ignored_buoys:
    buoy_datas.pop(station_id, None)

len_cleaned_data = len([data for data in buoy_datas.values() if 'Cleaned Marine' in data and 'Cleaned Meteo' in data])

# Résumé final du nettoyage
print("\n📊 RÉSUMÉ DU NETTOYAGE:")
print(f"📌 Stations au départ : {stations_depart}")
print(f"✅ Stations nettoyées : {len_cleaned_data}")
print(f"🏁 Stations restantes après filtrage :")

for station_id, reason in ignored_buoys.items():
    print(f"🛑 Station {station_id} ignorée: {reason}")

print(f"\n🧹 Clés restantes dans buoy_datas après purge : {len(buoy_datas)} (attendu : {len_cleaned_data})")
